In [1]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from ROMS/Air Raid (Men-A-Vision) (PAL) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from ROMS/Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying amidar.bin from ROMS/Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/amidar.bin
copying assault.bin from ROMS/Assau

In [2]:
!pip install --user keras-rl
!pip install --user keras==2.2.4
!pip install --user Pillow
!pip install --user h5py
!pip install  gym keras-rl2 gym[atari]

     |████████████████████████████████| 40kB 3.3MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp37-none-any.whl size=48382 sha256=aa89ccb3bb8a6c96e4c3f51678dbef0f725d4347d2b39d15ae05b88d6edc1f30
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
     |████████████████████████████████| 317kB 4.1MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 61kB 3.1MB/s 


In [3]:
import numpy as np
import gym
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory



In [4]:
# Get the environment and extract the number of actions and the input shape.

WINDOW_LENGTH = 4
env = gym.make('Breakout-v0')
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n
print(nb_actions)
input_shape_ = env.observation_space.shape
env.unwrapped.get_action_meanings()

4


['NOOP', 'FIRE', 'RIGHT', 'LEFT']

In [5]:

def build_model(input_shape, nb_actions):

  model = Sequential()

  model.add(Convolution2D(32, (8, 8), strides=(4, 4), input_shape=input_shape))
  model.add(Activation('relu'))
  model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
  model.add(Activation('relu'))
  model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
  model.add(Activation('relu'))
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dense(nb_actions))
  model.add(Activation('linear'))
  return model

model = build_model(input_shape=((WINDOW_LENGTH,) + input_shape_), nb_actions=nb_actions)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 4, 51, 39, 32)     6176      
_________________________________________________________________
activation (Activation)      (None, 4, 51, 39, 32)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 24, 18, 64)     32832     
_________________________________________________________________
activation_1 (Activation)    (None, 4, 24, 18, 64)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 22, 16, 64)     36928     
_________________________________________________________________
activation_2 (Activation)    (None, 4, 22, 16, 64)     0         
_________________________________________________________________
flatten (Flatten)            (None, 90112)             0

In [6]:

def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=100000, window_length=4)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [7]:
dqn = build_agent(model, nb_actions)

def train():

  dqn.compile(Adam(lr=1e-3), metrics=['mae'])
  filename = 'policy.h5'
  dqn.fit(env,
          nb_steps=10000,
          log_interval=1000, visualize=False, verbose=2)
  # After training is done, we save the final weights one more time.
  dqn.save_weights(filename, overwrite=True)

train()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Training for 10000 steps ...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  298/10000: episode: 1, duration: 32.027s, episode steps: 298, steps per second:   9, episode reward:  2.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 1.540 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  517/10000: episode: 2, duration: 3.767s, episode steps: 219, steps per second:  58, episode reward:  1.000, mean reward:  0.005 [ 0.000,  1.000], mean action: 1.374 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  691/10000: episode: 3, duration: 3.042s, episode steps: 174, steps per second:  57, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.391 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  868/10000: episode: 4, duration: 3.065s, episode steps: 177, steps per second:  58, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.350 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 1101/10000: episode: 5, duration: 98.160s, episode steps: 233, steps per second:   2, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 1.545 [0.000, 3.000],  loss: 30588.102708, mae: 31.076556, mean_q: 39.243301, mean_eps: 0.905455
 1407/10000: episode: 6, duration: 280.168s, episode steps: 306, steps per second:   1, episode reward:  2.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 1.650 [0.000, 3.000],  loss: 0.390123, mae: 1.153952, mean_q: 1.412927, mean_eps: 0.887185
 1574/10000: episode: 7, duration: 152.866s, episode steps: 167, steps per second:   1, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.527 [0.000, 3.000],  loss: 0.207311, mae: 1.190111, mean_q: 1.496693, mean_eps: 0.865900
 1842/10000: episode: 8, duration: 243.626s, episode steps: 268, steps per second:   1, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 1.586 [0.000, 3.000],  loss: 0.220254, mae: 1.206919, mean_q: 1.497572

In [8]:
!pip install colabgymrender
from colabgymrender.recorder import Recorder

def play(env):
  dqn.load_weights('policy.h5')

  """recorde and visualization the game"""
  directory = './video'
  env = Recorder(env, directory)

  observation = env.reset()
  terminal = False
  while not terminal:
    action = env.action_space.sample()
    observation, reward, terminal, info = env.step(action)
  env.play()

play(env=env)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2121728/45929032 bytes (4.6%)5562368/45929032 bytes (12.1%)8953856/45929032 bytes (19.5%)12500992/45929032 bytes (27.2%)16072704/45929032 bytes (35.0%)19660800/45929032 bytes (42.8%)23019520/45929032 bytes (50.1%)26591232/45929032 bytes (57.9%)30105600/45929032 bytes (65.5%)33472512/45929032 bytes (72.9%)37036032/45929032 bytes (80.6%)40673280/45929032 bytes (88.6%)